In [0]:
# Copyright 2017 Google LLC.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
# Install deps
!pip install -q "tensor2tensor-dev==1.3.1.dev5" tf-nightly

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os

from tensor2tensor import problems
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import trainer_utils
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
from tensorflow.contrib.eager.python import tfe
tfe.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

# Setup some directories
data_dir = os.path.expanduser("~/t2t/data")
tmp_dir = os.path.expanduser("~/t2t/tmp")
train_dir = os.path.expanduser("~/t2t/train")
checkpoint_dir = os.path.expanduser("~/t2t/checkpoints")
tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)
tf.gfile.MakeDirs(checkpoint_dir)

# Translate from English to French with a pre-trained model

In [0]:
# Translation
enfr_problem = registry.problem("translate_enfr_wmt_small32k")
enfr_problem.generate_data(data_dir, tmp_dir) 

In [0]:
example = tfe.Iterator(enfr_problem.dataset(Modes.TRAIN, data_dir)).next()
inputs = [int(x) for x in example["inputs"].numpy()] # Cast to ints.
targets = [int(x) for x in example["targets"].numpy()] # Cast to ints.

encoders = enfr_problem.feature_encoders(data_dir)
def decode(integers):
  samples = encoders["inputs"].decode(np.squeeze(integers))
  return samples[:samples.find("<EOS>")]

# Example inputs as int-tensor.
print("Inputs, encoded:")
print(inputs)
print("Inputs, decoded:")
# Example inputs as a sentence.
print(decode(inputs))
# Example targets as int-tensor.
print("Targets, encoded:")
print(targets)
# Example targets as a sentence.
print("Targets, decoded:")
print(decode(targets))

In [0]:
# Create hparams and the T2TModel object.
model_name = "transformer"
hparams_set = "transformer_base"

hparams = trainer_utils.create_hparams(hparams_set, data_dir)
hparams.use_eager_mode = True
trainer_utils.add_problem_hparams(hparams, "translate_enfr_wmt32k")

# NOTE: Only create the model once when restoring from a checkpoint; it's a
# Layer and so subsequent instantiations will have different variable scopes
# that will not match the checkpoint.
model = registry.model(model_name)(hparams, Modes.PREDICT)

In [0]:
# Copy the pretrained checkpoint locally
gs_ckpt_dir = "gs://tensor2tensor-checkpoints/"
ckpt_name = "transformer_enfr_test"
gs_ckpt = os.path.join(gs_ckpt_dir, ckpt_name)
local_ckpt = os.path.join(checkpoint_dir, ckpt_name)
!gsutil -q cp -R {gs_ckpt} {local_ckpt}
ckpt_path = tf.train.latest_checkpoint(local_ckpt)
ckpt_path

In [0]:
# Restore and translate!

def encode(input_str):
  # Encode from raw string to ints using problem encoders.
  inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
  batch_inputs = tf.reshape(inputs, [1, -1, 1, 1])  # Make it 4D.
  # TODO: rm target_space_id
  features_dict = {"inputs": batch_inputs,
                   "target_space_id": tf.constant(hparams.problems[0].target_space_id)}
  return features_dict

# Input to the decoder.
inputs = "This is a cat."

store = tfe.EagerVariableStore()
# Restore from checkpoint and run inference
with store.as_default():
  with tfe.restore_variables_on_create(ckpt_path):
    samples = model.infer(encode(inputs), beam_size=1)

print("Inputs: %s" % inputs)
print("Outputs: %s" % decode(samples))

# Train a custom model on MNIST

In [0]:
# Lots of problems available
problems.available()

In [0]:
# Create the MNIST problem and generate the data

mnist_problem = problems.problem("image_mnist")
# Generate data
mnist_problem.generate_data(data_dir, tmp_dir)

In [0]:
# Get the tf.data.Dataset from Problem.dataset
mnist_example = tfe.Iterator(mnist_problem.dataset(Modes.TRAIN, data_dir)).next()
image = mnist_example["inputs"]
label = mnist_example["targets"]

plt.imshow(image.numpy()[:, :, 0].astype(np.float32), cmap=plt.get_cmap('gray'))
print("Label: %d" % label.numpy())

In [0]:
# Lots of models available
registry.list_models()

In [0]:
# Create your own model

class MySimpleModel(t2t_model.T2TModel):

  def model_fn_body(self, features):
    inputs = features["inputs"]
    filters = self.hparams.hidden_size
    h1 = tf.layers.conv2d(inputs, filters,
                          kernel_size=(5, 5), strides=(2, 2))
    h2 = tf.layers.conv2d(tf.nn.relu(h1), filters,
                          kernel_size=(5, 5), strides=(2, 2))
    return tf.layers.conv2d(tf.nn.relu(h2), filters,
                            kernel_size=(3, 3))

hparams = trainer_utils.create_hparams("basic_1", data_dir)
hparams.hidden_size = 64
hparams.use_eager_mode = True
trainer_utils.add_problem_hparams(hparams, "image_mnist")
model = MySimpleModel(hparams, Modes.TRAIN)

In [0]:
# Train
store = tfe.EagerVariableStore()
optimizer = tf.train.AdamOptimizer()

# In Eager mode, opt.minimize must be passed a function that produces the loss
def loss_function(features):
  _, losses = model(features)
  return losses["training"]

tfe_loss_fn = tfe.implicit_value_and_gradients(loss_function)

NUM_STEPS = 500
BATCH_SIZE = 128

# Repeat and batch the data
mnist_train_dataset = mnist_problem.dataset(Modes.TRAIN, data_dir)
mnist_train_dataset = mnist_train_dataset.repeat(None).batch(BATCH_SIZE)

# Training loop
for count, example in enumerate(tfe.Iterator(mnist_train_dataset)):
  if count >= NUM_STEPS:
    break

  example["targets"] = tf.reshape(example["targets"], [BATCH_SIZE, 1, 1, 1])  # Make it 4D.
  loss, gv = tfe_loss_fn(example)
  optimizer.apply_gradients(gv)
  if count % 50 == 0:
    print("Step: %d, Loss: %.3f" % (count, loss.numpy()))

In [0]:
model.set_mode(Modes.EVAL)
mnist_eval_dataset = mnist_problem.dataset(Modes.EVAL, data_dir)
all_perplexities = []
all_accuracies = []
for count, example in enumerate(tfe.Iterator(mnist_eval_dataset)):
  if count >= 100:
    break

  batch_inputs = tf.reshape(example["inputs"], [1, 28, 28, 3])  # Make it 4D.
  batch_targets = tf.reshape(example["targets"], [1, 1, 1, 1])  # Make it 4D.
  features = {"inputs": batch_inputs, "targets": batch_targets}

  # Call the model.
  with store.as_default():
    predictions, _ = model(features)

  # Calculate and append the metrics
  all_perplexities.extend(metrics.padded_neg_log_perplexity(predictions, features["targets"]))
  all_accuracies.extend(metrics.padded_accuracy(predictions, features["targets"]))

# Print out metrics on the dataset
print("Accuracy: %.2f" % tf.reduce_mean(tf.concat(all_accuracies, axis=1)).numpy())